In [ ]:
import sys
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from transformers import LlamaTokenizer, AutoModelForCausalLM

In [ ]:
df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
sub = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/sample_submission.csv")

df['instruction'] = df['prompt'] + ' A: ' + df['A'] + ' B: ' + df['B'] + ' C: ' + df['C'] + ' D: ' + df['D'] + ' E: ' + df['E']
df

In [ ]:
model_name = '/kaggle/input/llama2-7b-stem'

tokenizer = LlamaTokenizer.from_pretrained(model_name)
    
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

In [ ]:
prompt = "<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n</s> [INST] query [/INST] "

preds = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    inputs = tokenizer(prompt.replace('query', row['instruction']) , return_tensors="pt").to(f"cuda:{model.device.index}")
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=1024, 
                                return_dict_in_generate=True, output_scores=True)

    first_token_probs = output.scores[0][0]
    option_scores = first_token_probs[[319, 350, 315, 360, 382]].float().cpu().numpy() #ABCDE
    pred = np.array(["A", "B", "C", "D", "E"])[np.argsort(option_scores)[::-1][:3]]
    pred = ' '.join(pred)
    preds.append(pred)
    
sub['prediction'] = preds

In [ ]:
sub.to_csv('submission.csv', index=False)
sub